# 교차 검증과 그리드 서치

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 검증 세트

In [5]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine-date')

In [6]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [7]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [8]:
# 문제 제기
# 이런 저런 값으로 모델을 많이 만들어서 
# 테스트 세트로 평가하면 결국 테스트 세트에 잘 맞는 모델이 만들어지는 것 아니냐?

# 지금까지 우리는 훈련 세트에서 모델을 훈련하고 테스트 세트에서 모델을 평가했다.
# 테스트 세트에서 얻은 점수를 보고 "실전에 투입하면 이정도 성능을 기대할 수 있겠다"고
# 생각했기 때문이었다.

# 그런데 테스트 세트를 사용해 자꾸 성능을 확인하다 보면
# 점점 테스트 세트에 맞추게 되는 셈..

# 테스트 세트로 일반화 성능을 올바르게 예측하려면, 가능한 한
# 테스트 세트를 사용하지 말아야 한다. 모델 만들고, 마지막에 딱 한번만 사용하는 게 좋다.
# 그러나 결정 트리는 테스트해볼만 매개변수가 많고,
# max_depth 매개변수를 사용한 하이퍼 파라미터 튜닝을 어케함?

# 테스트 세트를 사용하지 않고 이를 측정하는 방법 = 훈련 세트를 또 나누는 것 
# = 검증 세트

# 훈련 세트중에서 20%를 더 떼어 내어 검증(val) 세트로
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [9]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [10]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
# 훈련세트에 과도하게 적합되어 있어 매개변수를 바꿔서 더 좋은 모델을 찾아야 된다.

0.9971133028626413
0.864423076923077


## 교차 검증

In [11]:
from sklearn.model_selection import cross_validate # 교차검증
# 검증 세트를 만드느라 훈련 세트가 줄었다. 보통 많은 데이터를 훈련에 사용할수록
# 좋은 모델이 만들어진다. 그렇다고 검증 세트를 너무 조금 떼어 놓으면 검증 점수가
# 들쭊날쭉하고 불안정 - 이럴 떄 교차검증 이용해서 안정적인 검증 점수를 얻고 훈련에
# 더 많은 데이터를 사용할 수 있다.
# 교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러번 반복.
# 그 다음 이 점수를 평균하여 최종 검증 점수를 얻는다.
# ex) K-폴드 교차 검증

# 보통 5 나 10 폴드 교차 검증 많이 사용함. => 8~90%까지 훈련할 수 있다.
# 검증 세트는 줄어들겟지만, 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된
# 점수로 생각할 수 있다. 사이킷런에는 cross_validate()라는 교차 검증 함수가 있고,
# 먼저 평가할 모델 객체를 첫번째 매개 변수로 전달하고, 직접 검증 세트를 떼어내지 않고
# 훈련 세트 전체를 cross_validate()함수에 전달.
# 이 함수는 fit_time, score_time, test_score라는 키를 가진 딕셔너리 반환.
scores = cross_validate(dt, train_input, train_target)
# 이미 train_test_split에서 섞어서 여기서는 안섞음.
print(scores)
# 기본이 5폴드 교차라 5개 값씩 나온 것.

{'fit_time': array([0.00903749, 0.00704408, 0.00602269, 0.00605464, 0.00597143]), 'score_time': array([0.00191879, 0.00097394, 0.00090933, 0.00100255, 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [12]:
import numpy as np

print(np.mean(scores['test_score'])) # *검증 폴드의 점수 평균

0.855300214703487


In [13]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
# 굳이 한번 더 데이터들을 섞고 검증하고자한다면, cv에 k폴드 분할기를 사용
print(np.mean(scores['test_score']))

0.855300214703487


In [14]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
# n_splits는 몇 폴드 교차 검증을 할지 정하는.
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝
: 머신러닝 모델이 학습하는 파라미터를 모델파라미터라고 부르고, 모델이 학습할 수 없어서
사용자가 지정해야만 하는 파라미터를 하이퍼 파라미터라고 부른다.
이런 하이퍼 파라미터는 모두 클래스나 메서드의 매개변수로 표현된다.

하이퍼 파라미터를 튜닝하는 작업은 라이브러리가 기본으로 제공하는 기본값을 그대로 사용해 모델을 훈련하고, 검증 세트의 점수나 교차검증을 통해서 매개변수를 조금씩 바꿔본다.
모델마다 적게는 1~2개, 많게는 5~6개 매개변수를 제공.

매개변수들을 바꿔가면서 모델을 훈련하고 교차검증해야겠지

근데, ex) 결정 트리 모델에서 최적의 max_depth를 찾았다고 해서 다른 매개 변수(ex min_impurity_decrease)의 최적값을 찾기위해 매개변수를 바꿔가면 max_depth 값 역시 바뀐다.

매개변수가 많아질수록 문제가 더 복잡해진다.

사이킷런에서 제공하는 '그리드 서치'를 사용하면 친절하게도 하이퍼 파라미터 탐색과 교차검증을 한번에 수행하기에 별도로 cross_validate()함수를 호출할 필요X

In [15]:
from sklearn.model_selection import GridSearchCV
# min_impurity_decrease라는 파라미터 0.0001씩 올려가며 검증
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [16]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
# 그리드 서치의 cv 매개변수 기본값이 5라서 5폴드 교차 검증임.
# 따라서 파라미터 5개 검증 x 5 폴드 = 25개의 모델 훈련.

In [17]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [18]:
dt = gs.best_estimator_ # 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서
# 자동으로 다시 모델을 훈련. 이 모델은 gs 객체의 best_estimator 속성에 저장되어있다.

print(dt.score(train_input, train_target))

0.9615162593804117


In [19]:
print(gs.best_params_) # 그리드 서치로 찾은 최적의 매개 변수는 best_params_에
                        # 저장되어있다.

{'min_impurity_decrease': 0.0001}


In [20]:
print(gs.cv_results_['mean_test_score']) # 교차 검증의 평균점수

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [40]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(best_index)
# print(gs.cv_results_['params'])

# 수동으로 고르는 것보다 넘파이 argmax함수를 사용-> 가장 큰 인덱스 추출
print(gs.cv_results_['params'][best_index])

25
{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}


이 과정을 정리

1.탐색활 매개 변수 지정
2.훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다. 이 조합은 그리드 서치 객체에 저장된다.
3.그리드 서치는 최상의 매개변수에서 (교차검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해 최종 모델을 훈련합니다. 이 모델도 그리드 서치 객체에 저장된다.


In [22]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }
# 첫번째 파라미터는 총 9개
# 2 - 15개
# 3 - 10개

# 따라서 이 세개의 매개변수로 수행할 교차 검증 횟수는 9 * 15 * 10 = 1350개
# 근데 기본 5폴드교차검증이므로 모델의 수 1350 * 5 = 6750개


In [23]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [24]:
print(gs.best_params_) # 최상의 매개변수 조합

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [25]:
print(np.max(gs.cv_results_['mean_test_score'])) # 최상의 교차 검증 평균 점수

0.8683865773302731


### 랜덤 서치
: 매개 변수의 값이 수치일 때, 값의 범위나 간격을 미리 정하기 어려울 수 있다.
또 너무 많은 매개 변수 조건이 있어 그리드 서치 수행 시간이 오래걸릴 수 있는데,
이럴 때 랜덤 서치를 사용하면 좋다.

랜덤 서치에는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달.

In [26]:
from scipy.stats import uniform, randint # uniform은 실수값을 뽑는

In [27]:
rgen = randint(0, 10)
rgen.rvs(10)

array([1, 0, 6, 9, 1, 8, 7, 6, 2, 1], dtype=int64)

In [28]:
np.unique(rgen.rvs(1000), return_counts=True) # 1000개를 뽑지만, unique로 출력하면.. 

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([ 95, 110,  86, 103,  99,  98,  90, 112, 108,  99], dtype=int64))

In [29]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.28716872, 0.88256579, 0.97534441, 0.40338711, 0.44985621,
       0.32369116, 0.99391527, 0.27567314, 0.37843433, 0.22388152])

In [30]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25), 
          'min_samples_leaf': randint(1, 25), # 리프 노드가 되기 위한 최소 샘플 갯수
          }

In [31]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42) # n_iter = 샘플링 횟수
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001EA5C55CBE0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001EA5C55E3B0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001EA5C55D8A0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001EA5C55CAF0>},
                   random_state=42)

In [32]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [33]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [34]:
dt = gs.best_estimator_ # 최적의 모델은 이미 전체 훈련 세트로 훈련되어
                        # best_estimator 속성에 저장되어 있다.
                        # 이 모델을 최종 모델로 결정하고 테스트 세트의 성능을 확인하자.

print(dt.score(test_input, test_target))

0.86


## 확인문제

In [35]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001EA5C55CBE0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001EA5C55E3B0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001EA5C55D8A0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001EA5C55CAF0>},
                   random_state=42)

In [36]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077


In [41]:
print(test_input) # 맞네 처음부터 쓰던 와인 데이터들이네

[[12.2  12.8   3.26]
 [ 9.9   2.2   3.27]
 [12.    7.4   3.18]
 ...
 [12.4   1.8   3.19]
 [ 9.4   9.7   3.3 ]
 [ 8.7  15.5   2.9 ]]
